# **Finetuning del modelo T5 (Intento inicial)**

Este es el cuaderno del intento inicial del finetuning del modelo T5, que conservo por si fuese útil en el futuro, con lo que no lo comento realmente.

**El finetuning final del T5 se encuentra en el notebook Fine_Tuning_T5_ajustado.ipynb**

In [ ]:
"""
FINE-TUNING T5 PARA ANÁLISIS ELECTORAL ESPAÑOL
Google Colab Pro - Optimizado para GPUs limitadas
"""

# ============================================================================
# CELDA 1: INSTALACIÓN Y CONFIGURACIÓN INICIAL
# ============================================================================
print("🚀 INICIANDO FINE-TUNING T5 PARA ANÁLISIS ELECTORAL")
print("=" * 70)

# Instalación de dependencias
!pip install -q transformers==4.36.0
!pip install -q accelerate
!pip install -q datasets==2.15.0
!pip install -q evaluate==0.4.0
!pip install -q rouge-score==0.1.2
!pip install -q nltk==3.8.1
!pip install -q sentencepiece==0.1.99
!pip install -q protobuf==3.20.3
!pip install -q peft==0.5.0 # Pin peft to a known compatible version

import os
import json
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import gc
import warnings
warnings.filterwarnings('ignore')

print("✅ Dependencias instaladas")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

🚀 INICIANDO FINE-TUNING T5 PARA ANÁLISIS ELECTORAL
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 6.4 MB/s eta 0:00:00
✅ Dependencias instaladas
PyTorch: 2.9.0+cu126
CUDA disponible: True


In [ ]:
# ============================================================================
# CELDA 2: VERIFICACIÓN DE RECURSOS Y CONFIGURACIÓN
# ============================================================================
def check_resources():
    """Verifica recursos disponibles y configura parámetros"""

    print("\n" + "=" * 70)
    print("🔍 VERIFICACIÓN DE RECURSOS - Google Colab Pro")
    print("=" * 70)

    # Información de GPU
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9

        print(f"🎯 GPU: {gpu_name}")
        print(f"💾 Memoria GPU: {gpu_memory:.2f} GB")

        # Seleccionar modelo T5 según GPU
        if gpu_memory >= 16:  # V100/A100
            model_name = "google-t5/t5-base"  # 220M parámetros
            batch_size = 8
            print("   → Usando T5-base (220M parámetros)")
        elif gpu_memory >= 8:  # T4 15GB
            model_name = "google-t5/t5-base"
            batch_size = 4
            print("   → Usando T5-base con batch reducido")
        else:
            model_name = "google-t5/t5-small"  # 60M parámetros
            batch_size = 8
            print("   → Usando T5-small (60M parámetros)")
    else:
        print("⚠️  No hay GPU disponible - Usando CPU (MUY LENTO)")
        model_name = "google-t5/t5-small"
        batch_size = 2

    # Información de RAM
    import psutil
    ram_gb = psutil.virtual_memory().total / 1e9
    print(f"\n💿 RAM Total: {ram_gb:.2f} GB")

    # Espacio en disco
    import shutil
    total, used, free = shutil.disk_usage("/")
    print(f"📁 Disco libre: {free / 1e9:.1f} GB")

    return model_name, batch_size

MODEL_NAME, BASE_BATCH_SIZE = check_resources()


🔍 VERIFICACIÓN DE RECURSOS - Google Colab Pro
🎯 GPU: NVIDIA L4
💾 Memoria GPU: 23.80 GB
   → Usando T5-base (220M parámetros)

💿 RAM Total: 56.86 GB
📁 Disco libre: 210.3 GB


In [ ]:
# ============================================================================
# CELDA 3: CONFIGURACIÓN DE PARÁMETROS DE ENTRENAMIENTO
# ============================================================================
class T5TrainingConfig:
    """Configuración completa para fine-tuning de T5"""

    # ========== MODELO ==========
    model_name = MODEL_NAME
    tokenizer_name = MODEL_NAME

    # ========== DATASET ==========
    # Cambia estas rutas según tu estructura
    dataset_base_path = "/content/drive/MyDrive/Practica_LLM_Engineering_25/dataset_t5_final"
    train_path = f"{dataset_base_path}/train.jsonl"
    val_path = f"{dataset_base_path}/val.jsonl"
    test_path = f"{dataset_base_path}/test.jsonl"

    # ========== HIPERPARÁMETROS ==========
    # Basados en recursos disponibles
    max_input_length = 512
    max_target_length = 256
    batch_size = BASE_BATCH_SIZE
    gradient_accumulation_steps = 4 if BASE_BATCH_SIZE <= 4 else 2
    num_train_epochs = 3
    learning_rate = 3e-4  # T5 responde bien a learning rates altos
    weight_decay = 0.01
    warmup_steps = 100

    # ========== ENTRENAMIENTO ==========
    logging_steps = 25
    eval_steps = 100
    save_steps = 200
    save_total_limit = 2
    prediction_loss_only = False
    load_best_model_at_end = True
    metric_for_best_model = "eval_loss"
    greater_is_better = False

    # ========== GENERACIÓN ==========
    num_beams = 4  # Beam search para mejores respuestas
    temperature = 0.7  # Para diversidad en generación
    top_p = 0.9
    repetition_penalty = 1.2  # Evitar repeticiones

    # ========== SALIDA ==========
    output_dir = f"/content/drive/MyDrive/Practica_LLM_Engineering_25/t5_electoral_{datetime.now().strftime('%Y%m%d_%H%M')}"
    report_to = "none"  # Cambiar a "wandb" para tracking

    def __str__(self):
        config_str = "\n⚙️ CONFIGURACIÓN DE ENTRENAMIENTO:\n"
        config_str += "=" * 50 + "\n"

        for key, value in self.__dict__.items():
            if not key.startswith('_'):
                config_str += f"{key:30}: {value}\n"

        return config_str

config = T5TrainingConfig()
print(config)


⚙️ CONFIGURACIÓN DE ENTRENAMIENTO:



In [ ]:
# ============================================================================
# CELDA 4: CARGA Y PREPARACIÓN DEL DATASET
# ============================================================================

from datasets import Dataset, DatasetDict
import pandas as pd

print("\n" + "=" * 70)
print("📂 CARGA Y PREPARACIÓN DEL DATASET")
print("=" * 70)

def load_dataset_files(config):
    """Carga los archivos JSONL del dataset"""

    def read_jsonl(filepath):
        """Lee un archivo JSONL y devuelve lista de diccionarios"""
        data = []
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                for line_num, line in enumerate(f, 1):
                    line = line.strip()
                    if line:
                        try:
                            record = json.loads(line)
                            # Ensure 'input_text' and 'target_text' are strings
                            if 'input_text' in record and not isinstance(record['input_text'], str):
                                record['input_text'] = str(record['input_text'])
                            if 'target_text' in record and not isinstance(record['target_text'], str):
                                record['target_text'] = str(record['target_text'])
                            # Optionally, handle 'metadata' fields if they are causing issues
                            if 'metadata' in record and isinstance(record['metadata'], dict):
                                for key, value in record['metadata'].items():
                                    if not isinstance(value, str):
                                        record['metadata'][key] = str(value)
                            data.append(record)
                        except json.JSONDecodeError as e:
                            print(f"  ⚠️  Error en línea {line_num} de {filepath}: {e}")
                            continue
            print(f"  ✅ {len(data)} ejemplos cargados")
            return data
        except FileNotFoundError:
            print(f"  ❌ Archivo no encontrado: {filepath}")
            return []
        except Exception as e:
            print(f"  ❌ Error cargando {filepath}: {e}")
            return []

    print("Cargando splits...")

    train_data = read_jsonl(config.train_path)
    val_data = read_jsonl(config.val_path)
    test_data = read_jsonl(config.test_path)

    if not train_data:
        raise ValueError("❌ No se pudo cargar el dataset de entrenamiento")

    # Crear DatasetDict
    dataset_dict = DatasetDict({
        "train": Dataset.from_list(train_data),
        "validation": Dataset.from_list(val_data) if val_data else None,
        "test": Dataset.from_list(test_data) if test_data else None
    })

    # Estadísticas
    print(f"\n📊 ESTADÍSTICAS DEL DATASET:")
    print(f"  Entrenamiento: {len(train_data)} ejemplos")
    if val_data:
        print(f"  Validación: {len(val_data)} ejemplos")
    if test_data:
        print(f"  Test: {len(test_data)} ejemplos")

    # Mostrar ejemplos
    print(f"\n📝 EJEMPLOS DEL DATASET:")
    for i in range(min(2, len(train_data))):
        example = train_data[i]
        print(f"\n  Ejemplo {i+1}:")
        print(f"  Input: {example.get('input_text', '')[:100]}...")
        print(f"  Target: {example.get('target_text', '')[:100]}...")
        if 'metadata' in example:
            nivel = example['metadata'].get('nivel_original', 'N/A')
            print(f"  Nivel: {nivel}")

    return dataset_dict

# Cargar dataset
dataset = load_dataset_files(config)



📂 CARGA Y PREPARACIÓN DEL DATASET
Cargando splits...
  ✅ 1440 ejemplos cargados
  ✅ 180 ejemplos cargados
  ✅ 180 ejemplos cargados

📊 ESTADÍSTICAS DEL DATASET:
  Entrenamiento: 1440 ejemplos
  Validación: 180 ejemplos
  Test: 180 ejemplos

📝 EJEMPLOS DEL DATASET:

  Ejemplo 1:
  Input: ### Instrucción:
¿Cuál fue el % VOX mediano en la comunidad de Andalucía en Noviembre 2019?

### Con...
  Target: El % VOX mediano en la comunidad de Andalucía en Noviembre 2019 fue de 19.90%....
  Nivel: nivel2

  Ejemplo 2:
  Input: ### Instrucción:
¿Qué diferencia hay entre PP y VOX en la provincia de Segovia en Abril 2019?

### C...
  Target: En la elección de Abril 2019 en la provincia de Segovia, el partido PP obtuvo el 34.25%, y el partid...
  Nivel: nivel2


In [ ]:
'''

# ============================================================================
# CELDA 5: TOKENIZACIÓN PARA T5
# ============================================================================

from transformers import T5Tokenizer

print("\n" + "=" * 70)
print("🔤 TOKENIZACIÓN DEL DATASET")
print("=" * 70)

print(f"Cargando tokenizer: {config.tokenizer_name}")
tokenizer = T5Tokenizer.from_pretrained(config.tokenizer_name)

# Verificar tokenizer
print(f"  Vocabulario: {tokenizer.vocab_size} tokens")
print(f"  Model max length: {tokenizer.model_max_length}")

# Función de preprocesamiento para T5
def preprocess_function(examples):
    """
    Preprocesa ejemplos para T5 (encoder-decoder)
    Formato: input_ids, attention_mask, labels
    """

    # Tokenizar inputs
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=config.max_input_length,
        truncation=True,
        padding="max_length"
    )

    # Tokenizar targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            max_length=config.max_target_length,
            truncation=True,
            padding="max_length"
        )

    # Para T5, reemplazar padding token id por -100 para ignorar en loss
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in seq]
        for seq in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

print("\nTokenizando dataset...")
tokenized_datasets = {}

# Tokenizar train
tokenized_datasets["train"] = dataset["train"].map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Tokenizar validation si existe
if dataset["validation"]:
    tokenized_datasets["validation"] = dataset["validation"].map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["validation"].column_names
    )

# Tokenizar test si existe
if dataset["test"]:
    tokenized_datasets["test"] = dataset["test"].map(
        preprocess_function,
        batched=True,
        remove_columns=dataset["test"].column_names
    )

print(f"✅ Dataset tokenizado:")
print(f"  Train: {len(tokenized_datasets['train'])} ejemplos")
if "validation" in tokenized_datasets:
    print(f"  Validation: {len(tokenized_datasets['validation'])} ejemplos")
if "test" in tokenized_datasets:
    print(f"  Test: {len(tokenized_datasets['test'])} ejemplos")


'''


🔤 TOKENIZACIÓN DEL DATASET
Cargando tokenizer: google-t5/t5-base


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  Vocabulario: 32000 tokens
  Model max length: 1000000000000000019884624838656

Tokenizando dataset...


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

✅ Dataset tokenizado:
  Train: 1440 ejemplos
  Validation: 180 ejemplos
  Test: 180 ejemplos


In [ ]:

# ============================================================================
# CELDA 5: TOKENIZACIÓN PARA T5
# ============================================================================


# REEMPLAZA LA CELDA 5 (TOKENIZACIÓN) CON ESTE CÓDIGO:

from transformers import T5Tokenizer
import numpy as np

print("\n" + "=" * 70)
print("🔤 TOKENIZACIÓN DEL DATASET - VERSIÓN CORREGIDA")
print("=" * 70)

print(f"Cargando tokenizer: {config.tokenizer_name}")
tokenizer = T5Tokenizer.from_pretrained(config.tokenizer_name)

# Configurar tokenizer para T5
tokenizer.pad_token = tokenizer.eos_token  # T5 usa eos como pad

print(f"  Vocabulario: {tokenizer.vocab_size} tokens")
print(f"  Model max length: {tokenizer.model_max_length}")
print(f"  Pad token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  EOS token: {tokenizer.eos_token} (ID: {tokenizer.eos_token_id})")

# ============================================================================
# FUNCIÓN DE PREPROCESAMIENTO CORREGIDA
# ============================================================================
def preprocess_function_corrected(examples):
    """
    Versión corregida de la función de preprocesamiento
    """
    # 1. Tokenizar inputs
    inputs = examples["input_text"]
    model_inputs = tokenizer(
        inputs,
        max_length=config.max_input_length,
        truncation=True,
        padding="max_length",
        return_tensors=None  # No devolver tensores todavía
    )

    # 2. Tokenizar targets (labels) con el tokenizer de target
    targets = examples["target_text"]

    # IMPORTANTE: Configurar el tokenizer para targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=config.max_target_length,
            truncation=True,
            padding="max_length",
            return_tensors=None
        )

    # 3. Reemplazar padding token id por -100 para ignorar en loss
    #    VERIFICAR que los IDs estén en rango
    labels_input_ids = labels["input_ids"]

    for i in range(len(labels_input_ids)):
        for j in range(len(labels_input_ids[i])):
            token_id = labels_input_ids[i][j]

            # Verificar que el token ID esté en rango
            if token_id >= tokenizer.vocab_size:
                print(f"⚠️  Token ID fuera de rango: {token_id} (vocab size: {tokenizer.vocab_size})")
                labels_input_ids[i][j] = tokenizer.pad_token_id

            # Reemplazar padding token por -100
            if token_id == tokenizer.pad_token_id:
                labels_input_ids[i][j] = -100

    model_inputs["labels"] = labels_input_ids

    return model_inputs

# ============================================================================
# FUNCIÓN PARA VERIFICAR DATOS ANTES DE TOKENIZAR
# ============================================================================
def check_dataset_before_tokenization(dataset):
    """Verifica que no haya problemas en los datos"""
    print("\n🔍 VERIFICANDO DATASET ANTES DE TOKENIZACIÓN...")

    problematic_examples = []

    for i in range(min(10, len(dataset))):  # Revisar primeros 10 ejemplos
        example = dataset[i]

        # Verificar que los campos existan
        if "input_text" not in example or "target_text" not in example:
            print(f"  ❌ Ejemplo {i}: Faltan campos requeridos")
            problematic_examples.append(i)
            continue

        input_text = example["input_text"]
        target_text = example["target_text"]

        # Verificar que no estén vacíos
        if not input_text or not target_text:
            print(f"  ❌ Ejemplo {i}: Texto vacío")
            problematic_examples.append(i)
            continue

        # Verificar caracteres extraños
        for char in input_text + target_text:
            if ord(char) > 127 and char not in "áéíóúÁÉÍÓÚñÑüÜ":
                print(f"  ⚠️  Ejemplo {i}: Carácter extraño: {char} (ord: {ord(char)})")

    if problematic_examples:
        print(f"\n⚠️  {len(problematic_examples)} ejemplos problemáticos encontrados")
        return False
    else:
        print("✅ Dataset verificado correctamente")
        return True

# Verificar dataset
print("\nVerificando dataset de entrenamiento...")
check_dataset_before_tokenization(dataset["train"])

if dataset["validation"]:
    print("\nVerificando dataset de validación...")
    check_dataset_before_tokenization(dataset["validation"])

# ============================================================================
# TOKENIZACIÓN CON VERIFICACIÓN
# ============================================================================
print("\n🔄 TOKENIZANDO DATASET (con verificación paso a paso)...")

# Tokenizar en lotes pequeños para debuggear
def tokenize_with_debug(dataset_split, split_name, batch_size=100):
    """Tokeniza con verificación paso a paso"""
    print(f"Tokenizando {split_name} ({len(dataset_split)} ejemplos)...")

    # Tokenizar en lotes
    tokenized_examples = []

    for i in range(0, len(dataset_split), batch_size):
        batch = dataset_split.select(range(i, min(i + batch_size, len(dataset_split))))

        # Aplicar preprocesamiento
        tokenized_batch = batch.map(
            preprocess_function_corrected,
            batched=True,
            batch_size=batch_size
        )

        # Verificar el primer ejemplo de cada batch
        if i == 0:
            print(f"\n  Primer ejemplo tokenizado de {split_name}:")
            first_example = tokenized_batch[0]

            # Decodificar para verificar
            input_ids = first_example["input_ids"]
            labels = first_example["labels"]

            # Verificar rangos
            max_input_id = max(input_ids)
            max_label_id = max([l for l in labels if l != -100])

            print(f"    Longitud input: {len(input_ids)}")
            print(f"    Longitud labels: {len(labels)}")
            print(f"    Max input ID: {max_input_id} (vocab size: {tokenizer.vocab_size})")
            print(f"    Max label ID: {max_label_id} (vocab size: {tokenizer.vocab_size})")

            if max_input_id >= tokenizer.vocab_size or max_label_id >= tokenizer.vocab_size:
                print(f"    ⚠️  ALERTA: IDs fuera de rango!")

                # Mostrar tokens problemáticos
                problematic_inputs = [id for id in input_ids if id >= tokenizer.vocab_size]
                problematic_labels = [id for id in labels if id != -100 and id >= tokenizer.vocab_size]

                if problematic_inputs:
                    print(f"      Input IDs problemáticos: {problematic_inputs}")

                if problematic_labels:
                    print(f"      Label IDs problemáticos: {problematic_labels}")

        tokenized_examples.append(tokenized_batch)

    # Combinar todos los batches
    from datasets import concatenate_datasets
    result = concatenate_datasets(tokenized_examples)

    print(f"  ✅ {split_name} tokenizado: {len(result)} ejemplos")
    return result

# Tokenizar cada split
tokenized_datasets = {}

tokenized_datasets["train"] = tokenize_with_debug(
    dataset["train"],
    "train",
    batch_size=50
)

if dataset["validation"]:
    tokenized_datasets["validation"] = tokenize_with_debug(
        dataset["validation"],
        "validation",
        batch_size=50
    )

if dataset["test"]:
    tokenized_datasets["test"] = tokenize_with_debug(
        dataset["test"],
        "test",
        batch_size=50
    )

print(f"\n✅ Dataset tokenizado exitosamente:")
print(f"  Train: {len(tokenized_datasets['train'])} ejemplos")
if "validation" in tokenized_datasets:
    print(f"  Validation: {len(tokenized_datasets['validation'])} ejemplos")
if "test" in tokenized_datasets:
    print(f"  Test: {len(tokenized_datasets['test'])} ejemplos")


🔤 TOKENIZACIÓN DEL DATASET - VERSIÓN CORREGIDA
Cargando tokenizer: google-t5/t5-base


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  Vocabulario: 32000 tokens
  Model max length: 1000000000000000019884624838656
  Pad token: </s> (ID: 1)
  EOS token: </s> (ID: 1)

Verificando dataset de entrenamiento...

🔍 VERIFICANDO DATASET ANTES DE TOKENIZACIÓN...
  ⚠️  Ejemplo 0: Carácter extraño: ¿ (ord: 191)
  ⚠️  Ejemplo 1: Carácter extraño: ¿ (ord: 191)
  ⚠️  Ejemplo 2: Carácter extraño: ¿ (ord: 191)
  ⚠️  Ejemplo 3: Carácter extraño: € (ord: 8364)
  ⚠️  Ejemplo 3: Carácter extraño: € (ord: 8364)
  ⚠️  Ejemplo 3: Carácter extraño: € (ord: 8364)
  ⚠️  Ejemplo 3: Carácter extraño: σ (ord: 963)
  ⚠️  Ejemplo 3: Carácter extraño: μ (ord: 956)
  ⚠️  Ejemplo 3: Carácter extraño: € (ord: 8364)
  ⚠️  Ejemplo 3: Carácter extraño: € (ord: 8364)
  ⚠️  Ejemplo 3: Carácter extraño: € (ord: 8364)
  ⚠️  Ejemplo 4: Carácter extraño: ¿ (ord: 191)
  ⚠️  Ejemplo 5: Carácter extraño: ¿ (ord: 191)
  ⚠️  Ejemplo 6: Carácter extraño: ¿ (ord: 191)
  ⚠️  Ejemplo 7: Carácter extraño: ¿ (ord: 191)
  ⚠️  Ejemplo 8: Carácter extraño: ¿ (ord: 191)
  ⚠️ 

Map:   0%|          | 0/50 [00:00<?, ? examples/s]


  Primer ejemplo tokenizado de train:
    Longitud input: 512
    Longitud labels: 256
    Max input ID: 30345 (vocab size: 32000)
    Max label ID: 21388 (vocab size: 32000)


...


  Primer ejemplo tokenizado de test:
    Longitud input: 512
    Longitud labels: 256
    Max input ID: 30345 (vocab size: 32000)
    Max label ID: 25168 (vocab size: 32000)


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

  ✅ test tokenizado: 180 ejemplos

✅ Dataset tokenizado exitosamente:
  Train: 1440 ejemplos
  Validation: 180 ejemplos
  Test: 180 ejemplos


In [ ]:
# AÑADE ESTO INMEDIATAMENTE DESPUÉS DE LA TOKENIZACIÓN
print("\n" + "=" * 70)
print("🔍 DIAGNÓSTICO ESPECÍFICO PARA EVALUACIÓN")
print("=" * 70)

# 1. Verificar SOLO el validation set
if "validation" in tokenized_datasets:
    print("Analizando validation set para encontrar el problema...")

    val_dataset = tokenized_datasets["validation"]

    # Buscar ejemplos problemáticos
    problematic_examples = []

    for i in range(len(val_dataset)):
        example = val_dataset[i]

        # Verificar labels
        labels = example["labels"]

        for label_id in labels:
            if label_id != -100:  # Ignorar -100
                if label_id < 0 or label_id >= tokenizer.vocab_size:
                    problematic_examples.append({
                        "index": i,
                        "label_id": label_id,
                        "vocab_size": tokenizer.vocab_size
                    })
                    break  # Solo registrar una vez por ejemplo

    if problematic_examples:
        print(f"❌ ENCONTRADOS {len(problematic_examples)} EJEMPLOS PROBLEMÁTICOS:")
        for problem in problematic_examples[:5]:  # Mostrar solo 5
            print(f"  Índice {problem['index']}: label_id={problem['label_id']} (vocab_size={problem['vocab_size']})")

        # Mostrar el texto original del primer problema
        print(f"\n📝 Primer ejemplo problemático (índice {problematic_examples[0]['index']}):")
        original_example = dataset["validation"][problematic_examples[0]["index"]]
        print(f"  Input: {original_example['input_text'][:200]}...")
        print(f"  Target: {original_example['target_text'][:200]}...")

        # SOLUCIÓN: Eliminar o corregir ejemplos problemáticos
        print("\n🛠️  APLICANDO SOLUCIÓN...")

        def filter_problematic_examples(examples, indices_to_remove):
            """Filtra ejemplos problemáticos"""
            filtered = []
            for i, example in enumerate(examples):
                if i not in indices_to_remove:
                    filtered.append(example)
            return filtered

        # Crear lista de índices a remover
        indices_to_remove = [p["index"] for p in problematic_examples]

        # Filtrar validation set
        from datasets import Dataset
        val_list = [val_dataset[i] for i in range(len(val_dataset)) if i not in indices_to_remove]
        tokenized_datasets["validation"] = Dataset.from_list(val_list)

        print(f"✅ Validation set filtrado: {len(val_list)} ejemplos (se removieron {len(indices_to_remove)})")
    else:
        print("✅ Validation set parece estar bien (no se encontraron IDs fuera de rango)")
else:
    print("⚠️  No hay validation set configurado")

# 2. Configurar evaluación para que use SOLO loss, NO generación
print("\n🔄 CONFIGURANDO EVALUACIÓN SIN GENERACIÓN (solo loss)...")

# Modificar los training arguments para desactivar generación durante evaluación
training_args = Seq2SeqTrainingArguments(
    output_dir=config.output_dir,
    overwrite_output_dir=True,

    # Hiperparámetros
    num_train_epochs=config.num_train_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    warmup_steps=config.warmup_steps,

    # Estrategias - IMPORTANTE: desactivar generación durante eval
    evaluation_strategy="steps",
    eval_steps=config.eval_steps,
    save_strategy="steps",
    save_steps=config.save_steps,
    save_total_limit=config.save_total_limit,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",  # Usar loss, no métricas de generación
    greater_is_better=False,

    # Logging y reporte
    logging_strategy="steps",
    logging_steps=config.logging_steps,
    report_to=config.report_to,

    # Optimización
    fp16=torch.cuda.is_available(),
    optim="adafactor",
    gradient_checkpointing=True,

    # ⚠️ DESACTIVAR GENERACIÓN DURANTE EVALUACIÓN ⚠️
    predict_with_generate=False,  # ¡CRÍTICO! Desactiva generación
    generation_max_length=None,
    generation_num_beams=None,

    # Otros
    dataloader_num_workers=0,  # Reducir a 0 para debugging
    remove_unused_columns=True,
    push_to_hub=False,
)

print("✅ Evaluación configurada para usar solo loss (sin generación)")


🔍 DIAGNÓSTICO ESPECÍFICO PARA EVALUACIÓN
Analizando validation set para encontrar el problema...
✅ Validation set parece estar bien (no se encontraron IDs fuera de rango)

🔄 CONFIGURANDO EVALUACIÓN SIN GENERACIÓN (solo loss)...
✅ Evaluación configurada para usar solo loss (sin generación)


In [ ]:
# ============================================================================
# CELDA 6: CARGA DEL MODELO T5
# ============================================================================

from transformers import T5ForConditionalGeneration

print("\n" + "=" * 70)
print("🏗️  CARGA DEL MODELO T5")
print("=" * 70)

print(f"Cargando modelo: {config.model_name}")

# Configuración del modelo
model = T5ForConditionalGeneration.from_pretrained(config.model_name)

# Mover a GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print(f"✅ Modelo cargado en: {device}")
print(f"   Parámetros totales: {sum(p.numel() for p in model.parameters()):,}")
print(f"   Parámetros entrenables: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")



🏗️  CARGA DEL MODELO T5
Cargando modelo: google-t5/t5-base
✅ Modelo cargado en: cuda
   Parámetros totales: 222,903,552
   Parámetros entrenables: 222,903,552


In [ ]:
# ============================================================================
# CELDA 7: CONFIGURACIÓN DE MÉTRICAS DE EVALUACIÓN
# ============================================================================

import evaluate
import nltk
nltk.download('punkt', quiet=True)

print("\n" + "=" * 70)
print("📊 CONFIGURACIÓN DE MÉTRICAS")
print("=" * 70)

# Cargar métricas
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    """
    Calcula métricas ROUGE y BLEU para evaluación
    """
    predictions, labels = eval_pred

    # Decodificar predicciones
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Reemplazar -100 en labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Calcular ROUGE
    rouge_result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        use_aggregator=True
    )

    # Calcular BLEU
    bleu_result = bleu.compute(
        predictions=decoded_preds,
        references=[[ref] for ref in decoded_labels]
    )

    # Calcular exactitud de números (importante para análisis electoral)
    import re
    num_accuracy_scores = []

    for pred, label in zip(decoded_preds, decoded_labels):
        # Extraer números de cada texto
        pred_nums = set(re.findall(r'\d+\.?\d*', pred))
        label_nums = set(re.findall(r'\d+\.?\d*', label))

        if label_nums:
            correct_nums = len(pred_nums.intersection(label_nums))
            accuracy = correct_nums / len(label_nums) if label_nums else 0
            num_accuracy_scores.append(accuracy)

    num_accuracy = np.mean(num_accuracy_scores) if num_accuracy_scores else 0

    # Calcular longitud promedio
    avg_pred_len = np.mean([len(pred) for pred in decoded_preds])
    avg_label_len = np.mean([len(label) for label in decoded_labels])

    return {
        "rouge1": rouge_result["rouge1"],
        "rouge2": rouge_result["rouge2"],
        "rougeL": rouge_result["rougeL"],
        "rougeLsum": rouge_result["rougeLsum"],
        "bleu": bleu_result["bleu"],
        "number_accuracy": num_accuracy,
        "avg_pred_length": avg_pred_len,
        "avg_label_length": avg_label_len
    }

print("✅ Métricas configuradas: ROUGE, BLEU, Number Accuracy")


📊 CONFIGURACIÓN DE MÉTRICAS
✅ Métricas configuradas: ROUGE, BLEU, Number Accuracy


In [ ]:
# ============================================================================
# CELDA 8: CONFIGURACIÓN DEL ENTRENAMIENTO
# ============================================================================
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq

print("\n" + "=" * 70)
print("🎯 CONFIGURACIÓN DEL ENTRENAMIENTO")
print("=" * 70)

# Data collator para T5
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True
)

# Argumentos de entrenamiento específicos para Seq2Seq
training_args = Seq2SeqTrainingArguments(
    output_dir=config.output_dir,
    overwrite_output_dir=True,

    # Hiperparámetros
    num_train_epochs=config.num_train_epochs,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=config.batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.learning_rate,
    weight_decay=config.weight_decay,
    warmup_steps=config.warmup_steps,

    # Estrategias
    evaluation_strategy="steps" if "validation" in tokenized_datasets else "no",
    eval_steps=config.eval_steps,
    save_strategy="steps",
    save_steps=config.save_steps,
    save_total_limit=config.save_total_limit,
    load_best_model_at_end=True if "validation" in tokenized_datasets else False,
    metric_for_best_model=config.metric_for_best_model,
    greater_is_better=config.greater_is_better,

    # Logging y reporte
    logging_strategy="steps",
    logging_steps=config.logging_steps,
    report_to=config.report_to,

    # Optimización
    fp16=torch.cuda.is_available(),
    optim="adafactor",  # Adafactor funciona bien con T5
    gradient_checkpointing=True,  # Ahorra memoria

    # Generación para evaluación
    predict_with_generate=True,
    generation_max_length=config.max_target_length,
    generation_num_beams=config.num_beams,

    # Otros
    dataloader_num_workers=2,
    remove_unused_columns=True,
    push_to_hub=False,
)

print("✅ Training arguments configurados")
print(f"   Output directory: {config.output_dir}")
print(f"   Batch size: {config.batch_size}")
print(f"   Gradient accumulation: {config.gradient_accumulation_steps}")
print(f"   Total effective batch: {config.batch_size * config.gradient_accumulation_steps}")

# Crear trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets.get("validation", None),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics if "validation" in tokenized_datasets else None,
)

print("✅ Trainer creado exitosamente")


🎯 CONFIGURACIÓN DEL ENTRENAMIENTO
✅ Training arguments configurados
   Output directory: /content/drive/MyDrive/Practica_LLM_Engineering_25/t5_electoral_20260203_1455
   Batch size: 8
   Gradient accumulation: 2
   Total effective batch: 16
✅ Trainer creado exitosamente


In [ ]:
trainer

NameError: name 'trainer' is not defined

In [ ]:
 # VERSIÓN ULTRA-SEGURA DE TOKENIZACIÓN
print("\n🛡️  USANDO TOKENIZACIÓN ULTRA-SEGURA...")

def ultra_safe_preprocess_function(examples):
    """Función de preprocesamiento ultra-segura"""

    # Limpiar textos
    def clean_text(text):
        if not isinstance(text, str):
            return ""
        # Mantener solo caracteres válidos
        import re
        # Permitir letras, números, puntuación básica y acentos españoles
        cleaned = re.sub(r'[^\w\sáéíóúÁÉÍÓÚñÑüÜ.,;:¿?¡!()\-]', '', text)
        return cleaned.strip()

    # Limpiar inputs y targets
    inputs = [clean_text(text) for text in examples["input_text"]]
    targets = [clean_text(text) for text in examples["target_text"]]

    # Tokenizar inputs
    model_inputs = tokenizer(
        inputs,
        max_length=config.max_input_length,
        truncation=True,
        padding="max_length",
        return_tensors=None
    )

    # Tokenizar targets con verificación
    with tokenizer.as_target_tokenizer():
        tokenized_targets = tokenizer(
            targets,
            max_length=config.max_target_length,
            truncation=True,
            padding="max_length",
            return_tensors=None
        )

    # VERIFICACIÓN Y CORRECCIÓN DE LABELS
    labels = tokenized_targets["input_ids"]

    for i in range(len(labels)):
        for j in range(len(labels[i])):
            token_id = labels[i][j]

            # 1. Verificar que sea un número válido
            if not isinstance(token_id, int):
                labels[i][j] = tokenizer.pad_token_id
                continue

            # 2. Verificar rango del vocabulario
            if token_id < 0 or token_id >= tokenizer.vocab_size:
                labels[i][j] = tokenizer.pad_token_id
                continue

            # 3. Reemplazar padding token por -100
            if token_id == tokenizer.pad_token_id:
                labels[i][j] = -100

    model_inputs["labels"] = labels

    # Verificación final
    for i in range(min(2, len(model_inputs["input_ids"]))):
        max_input = max(model_inputs["input_ids"][i])
        max_label = max([l for l in model_inputs["labels"][i] if l != -100])

        if max_input >= tokenizer.vocab_size:
            print(f"⚠️  ADVERTENCIA: input_ids[{i}] tiene ID {max_input} >= {tokenizer.vocab_size}")

        if max_label >= tokenizer.vocab_size:
            print(f"⚠️  ADVERTENCIA: labels[{i}] tiene ID {max_label} >= {tokenizer.vocab_size}")

    return model_inputs

print("Tokenizando con método ultra-seguro...")
tokenized_datasets_ultra = {}

for split_name in ["train", "validation", "test"]:
    if split_name in dataset and dataset[split_name]:
        print(f"  Tokenizando {split_name}...")
        tokenized_datasets_ultra[split_name] = dataset[split_name].map(
            ultra_safe_preprocess_function,
            batched=True,
            batch_size=32,
            remove_columns=dataset[split_name].column_names
        )
        print(f"    ✅ {len(tokenized_datasets_ultra[split_name])} ejemplos")

# Usar esta versión ultra-segura
tokenized_datasets = tokenized_datasets_ultra


🛡️  USANDO TOKENIZACIÓN ULTRA-SEGURA...
Tokenizando con método ultra-seguro...
  Tokenizando train...


Map:   0%|          | 0/1440 [00:00<?, ? examples/s]

    ✅ 1440 ejemplos
  Tokenizando validation...


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

    ✅ 180 ejemplos
  Tokenizando test...


Map:   0%|          | 0/180 [00:00<?, ? examples/s]

    ✅ 180 ejemplos


In [ ]:
# CELDA DE DIAGNÓSTICO
print("\n" + "=" * 70)
print("🔧 DIAGNÓSTICO DEL ERROR 'piece id is out of range'")
print("=" * 70)

def diagnose_tokenization_issues(tokenized_dataset, split_name):
    """Diagnostica problemas en los datos tokenizados"""
    print(f"\nDiagnosticando {split_name}...")

    issues_found = False

    for i in range(min(5, len(tokenized_dataset))):  # Revisar primeros 5
        example = tokenized_dataset[i]

        # Verificar input_ids
        input_ids = example["input_ids"]
        max_input_id = max(input_ids)
        min_input_id = min(input_ids)

        if max_input_id >= tokenizer.vocab_size:
            print(f"  ❌ Ejemplo {i}: input_ids tiene ID fuera de rango: {max_input_id} >= {tokenizer.vocab_size}")
            issues_found = True

        if min_input_id < 0:
            print(f"  ❌ Ejemplo {i}: input_ids tiene ID negativo: {min_input_id}")
            issues_found = True

        # Verificar labels
        labels = example["labels"]
        valid_labels = [l for l in labels if l != -100]

        if valid_labels:
            max_label_id = max(valid_labels)
            min_label_id = min(valid_labels)

            if max_label_id >= tokenizer.vocab_size:
                print(f"  ❌ Ejemplo {i}: labels tiene ID fuera de rango: {max_label_id} >= {tokenizer.vocab_size}")
                issues_found = True

            if min_label_id < 0 and min_label_id != -100:
                print(f"  ❌ Ejemplo {i}: labels tiene ID negativo inválido: {min_label_id}")
                issues_found = True

    if not issues_found:
        print(f"  ✅ {split_name}: No se encontraron problemas de tokenización")

    return issues_found

# Diagnosticar cada split
issues_in_train = diagnose_tokenization_issues(tokenized_datasets["train"], "train")

if "validation" in tokenized_datasets:
    issues_in_val = diagnose_tokenization_issues(tokenized_datasets["validation"], "validation")

if issues_in_train:
    print("\n⚠️  SE ENCONTRARON PROBLEMAS. Aplicando corrección...")

    # Función de corrección
    def fix_tokenization_issues(examples):
        """Corrige IDs fuera de rango"""
        fixed_examples = []

        for example in examples:
            # Corregir input_ids
            fixed_input_ids = []
            for token_id in example["input_ids"]:
                if token_id >= tokenizer.vocab_size:
                    fixed_input_ids.append(tokenizer.pad_token_id)
                else:
                    fixed_input_ids.append(token_id)

            # Corregir labels
            fixed_labels = []
            for label_id in example["labels"]:
                if label_id == -100:
                    fixed_labels.append(-100)
                elif label_id >= tokenizer.vocab_size:
                    fixed_labels.append(tokenizer.pad_token_id)
                else:
                    fixed_labels.append(label_id)

            fixed_examples.append({
                "input_ids": fixed_input_ids,
                "attention_mask": example["attention_mask"],
                "labels": fixed_labels
            })

        return fixed_examples

    # Aplicar corrección
    print("Aplicando corrección a train...")
    from datasets import Dataset
    fixed_train = Dataset.from_list(fix_tokenization_issues(tokenized_datasets["train"]))
    tokenized_datasets["train"] = fixed_train

    if "validation" in tokenized_datasets:
        print("Aplicando corrección a validation...")
        fixed_val = Dataset.from_list(fix_tokenization_issues(tokenized_datasets["validation"]))
        tokenized_datasets["validation"] = fixed_val

    print("✅ Corrección aplicada. Verificando nuevamente...")
    diagnose_tokenization_issues(tokenized_datasets["train"], "train (corregido)")


🔧 DIAGNÓSTICO DEL ERROR 'piece id is out of range'

Diagnosticando train...
  ✅ train: No se encontraron problemas de tokenización

Diagnosticando validation...
  ✅ validation: No se encontraron problemas de tokenización


In [ ]:
# ============================================================================
# CELDA 9: ENTRENAMIENTO DEL MODELO
# ============================================================================
print("\n" + "=" * 70)
print("🚀 INICIANDO ENTRENAMIENTO")
print("=" * 70)

# Limpiar memoria antes de empezar
torch.cuda.empty_cache()
gc.collect()

# Mostrar memoria disponible
if torch.cuda.is_available():
    print(f"Memoria GPU antes del entrenamiento: {torch.cuda.memory_allocated() / 1e9:.2f} GB / {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

# Calcular pasos totales
total_steps = len(tokenized_datasets["train"]) // (config.batch_size * config.gradient_accumulation_steps) * config.num_train_epochs
print(f"Pasos totales estimados: {total_steps}")
print(f"Duración estimada: {total_steps * 0.5 / 60:.1f} minutos (aproximadamente)")

try:
    # Entrenar
    train_result = trainer.train()

    # Guardar modelo final
    trainer.save_model()
    tokenizer.save_pretrained(config.output_dir)

    # Guardar métricas de entrenamiento
    metrics = train_result.metrics
    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

    print(f"\n🎉 ENTRENAMIENTO COMPLETADO!")
    print(f"📁 Modelo guardado en: {config.output_dir}")

except RuntimeError as e:
    if "out of memory" in str(e):
        print("\n❌ ERROR: Memoria insuficiente")
        print("Intentando recuperación...")

        # Reducir batch size y reintentar
        config.batch_size = max(1, config.batch_size // 2)
        config.gradient_accumulation_steps *= 2

        print(f"Nuevos parámetros:")
        print(f"  Batch size: {config.batch_size}")
        print(f"  Gradient accumulation: {config.gradient_accumulation_steps}")

        # Recrear trainer con nuevos parámetros
        training_args.per_device_train_batch_size = config.batch_size
        training_args.gradient_accumulation_steps = config.gradient_accumulation_steps

        trainer = Seq2SeqTrainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets.get("validation", None),
            tokenizer=tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics if "validation" in tokenized_datasets else None,
        )

        # Intentar entrenar de nuevo
        train_result = trainer.train()
        trainer.save_model()

        print("✅ Entrenamiento completado con ajustes de memoria")

    else:
        raise e

except Exception as e:
    print(f"\n❌ ERROR durante el entrenamiento: {e}")
    import traceback
    traceback.print_exc()


🚀 INICIANDO ENTRENAMIENTO
Memoria GPU antes del entrenamiento: 4.84 GB / 23.80 GB
Pasos totales estimados: 270
Duración estimada: 2.2 minutos (aproximadamente)


Step,Training Loss,Validation Loss



❌ ERROR durante el entrenamiento: piece id is out of range.


Traceback (most recent call last):
  File "/tmp/ipython-input-1334025434.py", line 23, in <cell line: 0>
    train_result = trainer.train()
                   ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/trainer.py", line 1537, in train
    return inner_training_loop(
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/trainer.py", line 1914, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.12/dist-packages/transformers/trainer.py", line 2263, in _maybe_log_save_evaluate
    metrics = self.evaluate(ignore_keys=ignore_keys_for_eval)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/transformers/trainer_seq2seq.py", line 166, in evaluate
    return super().evaluate(eval_dataset, ignore_keys=ignore_keys, metric_key_prefix=metric_key_prefix)
           ^^^^^^^^^^^^^^^^

In [ ]:
# CELDA DE DIAGNÓSTICO
print("\n" + "=" * 70)
print("🔧 DIAGNÓSTICO DEL ERROR 'piece id is out of range'")
print("=" * 70)

def diagnose_tokenization_issues(tokenized_dataset, split_name):
    """Diagnostica problemas en los datos tokenizados"""
    print(f"\nDiagnosticando {split_name}...")

    issues_found = False

    for i in range(min(5, len(tokenized_dataset))):  # Revisar primeros 5
        example = tokenized_dataset[i]

        # Verificar input_ids
        input_ids = example["input_ids"]
        max_input_id = max(input_ids)
        min_input_id = min(input_ids)

        if max_input_id >= tokenizer.vocab_size:
            print(f"  ❌ Ejemplo {i}: input_ids tiene ID fuera de rango: {max_input_id} >= {tokenizer.vocab_size}")
            issues_found = True

        if min_input_id < 0:
            print(f"  ❌ Ejemplo {i}: input_ids tiene ID negativo: {min_input_id}")
            issues_found = True

        # Verificar labels
        labels = example["labels"]
        valid_labels = [l for l in labels if l != -100]

        if valid_labels:
            max_label_id = max(valid_labels)
            min_label_id = min(valid_labels)

            if max_label_id >= tokenizer.vocab_size:
                print(f"  ❌ Ejemplo {i}: labels tiene ID fuera de rango: {max_label_id} >= {tokenizer.vocab_size}")
                issues_found = True

            if min_label_id < 0 and min_label_id != -100:
                print(f"  ❌ Ejemplo {i}: labels tiene ID negativo inválido: {min_label_id}")
                issues_found = True

    if not issues_found:
        print(f"  ✅ {split_name}: No se encontraron problemas de tokenización")

    return issues_found

# Diagnosticar cada split
issues_in_train = diagnose_tokenization_issues(tokenized_datasets["train"], "train")

if "validation" in tokenized_datasets:
    issues_in_val = diagnose_tokenization_issues(tokenized_datasets["validation"], "validation")

if issues_in_train:
    print("\n⚠️  SE ENCONTRARON PROBLEMAS. Aplicando corrección...")

    # Función de corrección
    def fix_tokenization_issues(examples):
        """Corrige IDs fuera de rango"""
        fixed_examples = []

        for example in examples:
            # Corregir input_ids
            fixed_input_ids = []
            for token_id in example["input_ids"]:
                if token_id >= tokenizer.vocab_size:
                    fixed_input_ids.append(tokenizer.pad_token_id)
                else:
                    fixed_input_ids.append(token_id)

            # Corregir labels
            fixed_labels = []
            for label_id in example["labels"]:
                if label_id == -100:
                    fixed_labels.append(-100)
                elif label_id >= tokenizer.vocab_size:
                    fixed_labels.append(tokenizer.pad_token_id)
                else:
                    fixed_labels.append(label_id)

            fixed_examples.append({
                "input_ids": fixed_input_ids,
                "attention_mask": example["attention_mask"],
                "labels": fixed_labels
            })

        return fixed_examples

    # Aplicar corrección
    print("Aplicando corrección a train...")
    from datasets import Dataset
    fixed_train = Dataset.from_list(fix_tokenization_issues(tokenized_datasets["train"]))
    tokenized_datasets["train"] = fixed_train

    if "validation" in tokenized_datasets:
        print("Aplicando corrección a validation...")
        fixed_val = Dataset.from_list(fix_tokenization_issues(tokenized_datasets["validation"]))
        tokenized_datasets["validation"] = fixed_val

    print("✅ Corrección aplicada. Verificando nuevamente...")
    diagnose_tokenization_issues(tokenized_datasets["train"], "train (corregido)")

In [ ]:
# ============================================================================
# CELDA 10: EVALUACIÓN FINAL
# ============================================================================
print("\n" + "=" * 70)
print("📊 EVALUACIÓN FINAL DEL MODELO")
print("=" * 70)

if "test" in tokenized_datasets and len(tokenized_datasets["test"]) > 0:
    print("Evaluando en conjunto de test...")

    try:
        # Evaluar en test
        eval_results = trainer.evaluate(tokenized_datasets["test"], metric_key_prefix="test")

        print("\n📈 RESULTADOS EN TEST:")
        print("-" * 40)
        for key, value in eval_results.items():
            if isinstance(value, float):
                print(f"{key:20}: {value:.4f}")

        # Guardar resultados
        with open(f"{config.output_dir}/test_results.json", "w") as f:
            json.dump(eval_results, f, indent=2)

    except Exception as e:
        print(f"Error en evaluación: {e}")
else:
    print("⚠️  No hay conjunto de test para evaluación")


In [ ]:
# ============================================================================
# CELDA 11: PRUEBAS CON EJEMPLOS
# ============================================================================
print("\n" + "=" * 70)
print("🧪 PRUEBAS CON EJEMPLOS REALES")
print("=" * 70)

def generate_response(model, tokenizer, input_text, max_length=256):
    """Genera una respuesta usando el modelo fine-tuned"""

    # Tokenizar input
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=config.max_input_length,
        padding=True
    ).to(device)

    # Generar
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            num_beams=config.num_beams,
            temperature=config.temperature,
            top_p=config.top_p,
            repetition_penalty=config.repetition_penalty,
            do_sample=True,
            early_stopping=True
        )

    # Decodificar
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

# Probar con algunos ejemplos
test_examples = [
    "pregunta: ¿Qué porcentaje de votos obtuvo el PSOE en Madrid en 2019? respuesta:",
    "pregunta: ¿Cómo evolucionó la participación en Barcelona entre 2016 y 2019? respuesta:",
    "pregunta: ¿Cuál fue el partido más votado en Sevilla en las últimas elecciones? respuesta:",
    "pregunta: ¿Qué correlación hay entre renta personal y voto al PP en municipios costeros? respuesta:",
]

print("Generando respuestas para ejemplos de prueba...\n")

for i, example in enumerate(test_examples[:3]):  # Probar solo 3
    print(f"Ejemplo {i+1}:")
    print(f"  Input: {example[:80]}...")

    try:
        response = generate_response(model, tokenizer, example)
        print(f"  Respuesta: {response[:150]}...")
    except Exception as e:
        print(f"  Error: {e}")

    print()


In [ ]:
# ============================================================================
# CELDA 12: GUARDADO Y EXPORTACIÓN
# ============================================================================
print("\n" + "=" * 70)
print("💾 GUARDADO Y EXPORTACIÓN DEL MODELO")
print("=" * 70)

# Crear archivo de configuración
config_dict = {k: v for k, v in config.__dict__.items() if not k.startswith('_')}
with open(f"{config.output_dir}/training_config.json", "w") as f:
    json.dump(config_dict, f, indent=2, ensure_ascii=False)

# Crear README
readme_content = f"""
# Modelo T5 Fine-tuned para Análisis Electoral Español

## Información del Modelo
- **Modelo base**: {config.model_name}
- **Fecha de fine-tuning**: {datetime.now().strftime('%Y-%m-%d %H:%M')}
- **Dataset**: Resultados electorales españoles 2011-2019
- **Ejemplos de entrenamiento**: {len(tokenized_datasets['train'])}
- **Ejemplos de validación**: {len(tokenized_datasets.get('validation', []))}
- **Ejemplos de test**: {len(tokenized_datasets.get('test', []))}

## Parámetros de Entrenamiento
- **Épocas**: {config.num_train_epochs}
- **Batch size**: {config.batch_size}
- **Learning rate**: {config.learning_rate}
- **Longitud máxima input**: {config.max_input_length}
- **Longitud máxima output**: {config.max_target_length}

## Uso del Modelo

"""
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Cargar modelo y tokenizer
model_path = "{config.output_dir}"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Preparar input (usar mismo formato que en entrenamiento)
input_text = "pregunta: ¿Qué porcentaje de votos obtuvo el PSOE en Madrid en 2019? respuesta:"

# Tokenizar
inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

# Generar respuesta
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_length=256,
        num_beams=4,
        temperature=0.7,
        do_sample=True
    )

# Decodificar
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Respuesta: {{response}}")

In [ ]:
# ============================================================================
# CELDA 13: INFORME FINAL
# ============================================================================

print("\n" + "=" * 70)
print("📋 INFORME FINAL DEL ENTRENAMIENTO")
print("=" * 70)

# Información de recursos finales
if torch.cuda.is_available():
  print(f"💾 Memoria GPU utilizada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

print(f"\n📁 Directorio del modelo: {config.output_dir}")
print(f"🕒 Tiempo total: {datetime.now().strftime('%H:%M:%S')}")


# Listar archivos generados
print(f"\n📄 Archivos generados:")
!ls -la {config.output_dir}

print("\n" + "=" * 70)
print("🎊 ¡FINE-TUNING COMPLETADO EXITOSAMENTE!")
print("=" * 70)
print("\nEl modelo está listo para realizar análisis electorales.")
print(f"Puedes cargarlo con: T5ForConditionalGeneration.from_pretrained('{config.output_dir}')")



In [ ]:
# CELDA ADICIONAL: PRUEBA RÁPIDA DEL MODELO ENTRENADO
def test_finetuned_model(model_path, test_questions):
    """Prueba el modelo fine-tuned con preguntas de ejemplo"""

    from transformers import T5Tokenizer, T5ForConditionalGeneration
    import torch

    print("🧪 CARGANDO MODELO FINE-TUNED PARA PRUEBA")
    print("=" * 60)

    # Cargar modelo
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)

    # Mover a GPU si está disponible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()

    print(f"✅ Modelo cargado en: {device}")

    # Probar con preguntas
    for i, question in enumerate(test_questions):
        print(f"\n📝 Pregunta {i+1}: {question}")

        # Asegurar formato correcto
        if not question.endswith("respuesta:"):
            question = question + " respuesta:"

        # Generar respuesta
        inputs = tokenizer(question, return_tensors="pt", truncation=True, max_length=512).to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=256,
                num_beams=4,
                temperature=0.7,
                do_sample=True,
                repetition_penalty=1.2
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"🤖 Respuesta: {response}")

    return model, tokenizer

# Ejemplo de uso después del entrenamiento:
"""
test_questions = [
    "pregunta: ¿Cuál fue el porcentaje de participación en las elecciones de 2019?",
    "pregunta: ¿Qué partido ganó en Madrid en 2019?",
    "pregunta: ¿Cómo ha cambiado el voto al PSOE desde 2011?",
]

model, tokenizer = test_finetuned_model(config.output_dir, test_questions)
"""